# Deploy a custom classification model from Watson NLP as a Python function

This notebook shows the steps needed to create and deploy a Python function that classifies incoming documents. The classification is done by a custom model `classification_model_rt22_1`, which is part of this sample project. The example creates an _online deployment_ of the Python function. This way, the classification can be invoked via a web service in applications.

## Prerequisites 

Before running this notebook:
1. Create a new deployment space, or identify an existing deployment space to use for this example
2. Open the `Assets` tab of this sample project
3. Click the three-dot-menu for `classification_model_rt22_2` and select `Promote to space`
4. As target space, use the deployment space from step 1

## Initialize the Watson Machine Learning client

In [1]:
CPD_URL = '<CPD PLATFORM URL>'

In [2]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [3]:
wml_credentials = {
    "url": CPD_URL,
    "token": wslib.auth.get_current_token(),
    "instance_id": "wml_local",
    "version" : "4.6"
}

In [4]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [5]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------  ------------------------
ID                                    NAME     CREATED
de730b9a-b195-46f3-bddb-67084161959c  ChurnQA  2023-01-25T07:58:48.853Z
6a08ee38-e202-4c19-8580-99acfa3701ae  test     2022-09-14T06:40:42.288Z
------------------------------------  -------  ------------------------


### Add the `ID` of the space to use

In [6]:
client.set.default_space('<YOUR SPACE>')

'SUCCESS'

## Create a Python function that classifies documents

The model that is used here is the SVM model that is created in the example notebook `Complaint Classification`, section _Train a SVM classification model with Watson NLP_. The input data for _training_ the model was prepared in two steps:
1. Syntax Analysis
2. Creation of USE embeddings

Hence, for _applying_ the model on new data, the input data has to be transformed in the same way.

In [7]:
def detect_category():
    import watson_nlp
    
    # Syntax Model
    syntax_model = watson_nlp.load('syntax_izumo_en_stock')
    # USE Embedding Model
    use_model = watson_nlp.load('embedding_use_en_stock')
    
    #custom classification model
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()
    
    svm_model = watson_nlp.load(wslib.load_data('classification_model_rt22_2'))
      
    
    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]: 
            scoring_prediction_row = []
            for input_data in input_data_row:
                # run syntax model first
                syntax_result = syntax_model.run(input_data)
                # run custom model on top of syntax result
                svm_preds = svm_model.run(use_model.run(syntax_result, doc_embed_style='raw_text'))
                
                scoring_prediction_row.append(svm_preds.to_dict())
                
            scoring_prediction_out.append(scoring_prediction_row)
            
        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'], 
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response
    
    return score

## Save the Python function to the Model repository

In [9]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-22.2-py3.10")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'classification_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'classification_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


classification_function_details = client.repository.store_function(meta_props=meta_data, 
                                                                   function=detect_category)

## Create an Online Deployment

In [10]:
classification_function_uid = client.repository.get_function_uid(classification_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-classification-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-classification-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

classification_deployment_details = client.deployments.create(classification_function_uid, 
                                                              meta_props=meta_props)
classification_deployment_id = client.deployments.get_uid(classification_deployment_details)



#######################################################################################

Synchronous deployment creation for uid: '02581701-aeed-47e7-8728-e3b493512008' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.............................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1b98b361-b5fc-41bb-a171-f26ae57af6e6'
------------------------------------------------------------------------------------------------




## Classify sample complaints

### Download the complaint data

In [11]:
import pandas as pd
url = "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max=2021-03-30&date_received_min=2021-02-28&field=all&format=csv&has_narrative=true&no_aggs=true&size=18102"

complaint_df = pd.read_csv(url)
text_col = 'Consumer complaint narrative'

complaint_df.head(2)

Date received                                            Product  \
0      03/23/21                                           Mortgage   
1      03/23/21  Credit reporting, credit repair services, or o...   

                  Sub-product  \
0  Conventional home mortgage   
1            Credit reporting   

                                               Issue  \
0  Problem with a credit reporting company's inve...   
1               Incorrect information on your report   

                        Sub-issue  \
0                            None   
1  Personal information incorrect   

                        Consumer complaint narrative  \
0  After calling LoanCare to sort out a new escro...   
1  XXXX XXXX XXXX XXXX XXXX XXXX, Massachusetts X...   

                             Company public response  \
0  Company has responded to the consumer and the ...   
1  Company has responded to the consumer and the ...   

                                  Company State ZIP code  Tags  \
0                           LoanCare, LLC    CA    95409  None   
1  TRANSUNION INTERMEDIATE HOLDINGS, INC.    MA    02121  None   

  Consumer consent provided? Submitted via Date sent to company  \
0           Consent provided           Web             03/29/21   
1           Consent provided           Web             03/23/21   

  Company response to consumer Timely response?  Consumer disputed?  \
0      Closed with explanation              Yes                 NaN   
1      Closed with explanation              Yes                 NaN   

   Complaint ID  
0       4238609  
1       4239316

### Collect some sample complaints as input

In [12]:
complaints = complaint_df['Consumer complaint narrative'].values.tolist()

inputData = []
for i in range(1,30):
    inputData.append([complaints[i]])

In [13]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        'fields': ["Text"],
        'values': inputData
   }]
}

### Predict the complaint category

In [14]:
predictions = client.deployments.score(classification_deployment_id, scoring_payload)
predictions

{'predictions': [{'fields': ['nlp_prediction'],
   'values': [[{'classes': [{'class_name': 'Credit reporting, credit repair services, or other personal consumer reports',
        'confidence': 0.3759094476699829},
       {'class_name': 'Checking or savings account',
        'confidence': 0.07012385129928589},
       {'class_name': 'Credit card or prepaid card',
        'confidence': 0.04191317409276962},
       {'class_name': 'Money transfer, virtual currency, or money service',
        'confidence': 0.03531830385327339},
       {'class_name': 'Mortgage', 'confidence': 0.013855344615876675},
       {'class_name': 'Debt collection', 'confidence': 0.011920418590307236},
       {'class_name': 'Vehicle loan or lease',
        'confidence': 0.004009295254945755}],
      'producer_id': {'name': 'SVM classifier', 'version': '0.0.1'}}],
    [{'classes': [{'class_name': 'Credit reporting, credit repair services, or other personal consumer reports',
        'confidence': 0.8714433908462524},
   

## Clean up: Delete the deployment and the function

In [15]:
client.deployments.delete(classification_deployment_id)

'SUCCESS'

In [16]:
client.repository.delete(classification_function_uid)

'SUCCESS'